# Treinando em Python e inferindo em Java
## NeuralJava
![](./icone.png)
https://github.com/cleuton/neuraljava

**Cleuton Sampaio**

Não é preciso muito para concluir que **Java** é uma ótima opção para entregar aplicações em ambiente produtivo. É mais *limpa* e possui menos dependências *espalhadas* que outras linguagens de programação, como **Python** (que, por sinal, eu adoro). 
Mas temos que enxergar a realidade. Enquanto **Python** é excelente para desenvolvimento, prototipação e experimentação, especialmente em ciência de dados e IA, perde um pouco quando pensamos em ambiente produtivo. Por quê? 
- **Performance**: Além da questão do [**GIL**](https://wiki.python.org/moin/GlobalInterpreterLock), há vários *benchmarks* que comprovam o desempenho superior de aplicações Java, como este: https://benchmarksgame-team.pages.debian.net/benchmarksgame/fastest/python.html;
- **Robustez**: Com **Java**, temos pouca **entropia** de ambiente, isso quer dizer, menos dependências *espalhadas* e menos fragilidade. Basta que a JVM esteja instalada corretamente, que o resto das dependências pode ser *embutida* em um **uber jar** ou um **war**. Em outras linguagens, como **Python**, dependemos muito mais de um ambiente configurado, seja com **Anaconda** ou **virtualenv**, o que *espalha* as dependências para fora da sua aplicação;
- **Segurança**: Embora existam decompiladores **Java**, podemos [*obfuscar* o **bytecode**](https://www.owasp.org/index.php/Bytecode_obfuscation) e entregar apenas o *executável*. Em outras linguagens, como **Python**, ficamos limitados a entregar o código-fonte. É claro que podemos utilizar algo como **Cython** e gerar um executável, mas, nem todas as dependências são compatíveis com isto.

O objetivo desta demonstração é mostrar como é possível nos beneficiarmos das vantagens de ambas as ferramentas, **Java** e **Python**, preparando aplicações de **Deep Learning** de maneira fácil e robusta para entrarem em **Produção**.


## Estudo de caso
Vamos utilizar o dataset [**MNIST**](http://yann.lecun.com/exdb/mnist/) para reconhecer dígitos escritos manualmente. Na verdade, vamos usar o próprio modelo de exemplo da [biblioteca **Keras**](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py), de modo a não "reinventar a roda".

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
# Hiperparâmetros:
batch_size = 128
num_classes = 10
epochs = 12
img_rows, img_cols = 28, 28

In [3]:
# Carregando dados de treino e teste: 
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [11]:
# Preparando as imagens para o formato esperado pelo Keras: 
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
    print("Channels first")
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    print(K.image_data_format())

channels_last


In [5]:
# Formatando os valores de treino e teste e normalizando: 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
# Convertendo o rótulo em categorias (one-hot-encoding):
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
# Criação do modelo: 
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
# Compilação do modelo: 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [9]:
# Treinamento do modelo:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 77s 1ms/step - loss: 0.2654 - acc: 0.9188 - val_loss: 0.0592 - val_acc: 0.9815
Epoch 2/12
60000/60000 [==============================] - 76s 1ms/step - loss: 0.0925 - acc: 0.9733 - val_loss: 0.0386 - val_acc: 0.9867
Epoch 3/12
60000/60000 [==============================] - 73s 1ms/step - loss: 0.0675 - acc: 0.9798 - val_loss: 0.0352 - val_acc: 0.9881
Epoch 4/12
60000/60000 [==============================] - 73s 1ms/step - loss: 0.0559 - acc: 0.9832 - val_loss: 0.0320 - val_acc: 0.9900
Epoch 5/12
60000/60000 [==============================] - 74s 1ms/step - loss: 0.0478 - acc: 0.9858 - val_loss: 0.0284 - val_acc: 0.9908
Epoch 6/12
60000/60000 [==============================] - 74s 1ms/step - loss: 0.0410 - acc: 0.9875 - val_loss: 0.0273 - val_acc: 0.9915
Epoch 7/12
60000/60000 [==============================] - 74s 1ms/step - loss: 0

In [10]:
# Salvando o modelo em formato H5 - https://www.tinymind.com/learn/terms/hdf5:
model.save("meu_mnist.h5")

Agora, para continuar veja o projeto **Java** anexo a este notebook.